In [10]:
%matplotlib inline
from selenium import webdriver
from pandas import *
import pandas
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import *
import os
import pymysql
import time

In [2]:
#Data feature I used on stats.nba.com
#player bio name and team
#team clutch stats
#player clutch stats
#player box score
#shooting stats

In [2]:
#new chrome browser
browser = webdriver.Chrome()

In [44]:
#player bio name and team
url = 'https://stats.nba.com/players/traditional/'
browser.get(url)
option_list = [2,3,4,5,6]
season_list = ["2018-19","2017-18","2016-17","2015-16","2014-15"]
for season_id in range(len(option_list)):
    for season_option in range(2,4):
        season_path = "/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[1]/div/div/label/select/option[%d]" % option_list[season_id]
        browser.find_element_by_xpath(season_path).click()
        time.sleep(5)
        type_path = "/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[2]/div/div/label/select/option[%d]" % season_option
        browser.find_element_by_xpath(type_path).click()
        time.sleep(5)
        display_path = "/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]"
        browser.find_element_by_xpath(display_path).click()
        time.sleep(5)
        table = browser.find_element_by_class_name('nba-stat-table__overflow')
        
        seasonType = ''
        if season_option == 2:
            seasonType = "Regular_Season"
        else:
            seasonType = "Playoff"
        #parse table
        player_names = []
        player_team = []
        player_pts = []
        player_rebounds = []
        player_assists = []
        player_steals = []
        player_AvgRatings = []
        seasons = []
        season_type = []
        column_names = ['PlayerName', 'Team',"AvgPoints", "AvgRebounds", "AvgAssists", "AvgSteals", "AvgRatings","Season","SeasonType"]
        for line_id, lines in enumerate(table.text.split('\n')):
            if line_id != 0:
                if line_id % 3 == 2:
                    player_names.append(lines)
                if line_id % 3 == 0:
                    temp = lines.split(' ')
                    player_team.append(temp[0])
                    player_pts.append(temp[6])
                    player_rebounds.append(temp[-10])
                    player_assists.append(temp[-9])
                    player_steals.append(temp[-7])
                    player_AvgRatings.append(0.0)
                    seasons.append(season_list[season_id])
                    season_type.append(seasonType)

        #create dataframe
        player_bio_table = pandas.DataFrame({ column_names[0]: player_names,
                               column_names[1]: player_team,
                                             column_names[2]: player_pts,
                               column_names[3]: player_rebounds,
                                             column_names[4]: player_assists,
                               column_names[5]: player_steals,
                                             column_names[6]: player_AvgRatings,
                               column_names[7]: seasons,
                                             column_names[8]:season_type
                               }
                             )
        #write into mysql server
        conn = pymysql.connect(
            port=int(3306),
            user="root",
            passwd= "12345678",
            database = "NBA_DB"
        )
        if(season_id == 0 and season_option == 2):
            my_cursor = conn.cursor()
            my_cursor.execute("CREATE TABLE Player_Bio (PlayerName VARCHAR(255), TeamName VARCHAR(255), AvgPoints float, AvgRebounds float, AvgAssists float, AvgSteals float, AvgRating float, Season VARCHAR(255), SeasonType VARCHAR(255))")
        my_cursor = conn.cursor()
        sqlFormula = "INSERT INTO Player_Bio (PlayerName, TeamName, AvgPoints, AvgRebounds, AvgAssists, AvgSteals, AvgRating, Season, SeasonType) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
        for index, row in player_bio_table.iterrows():
            #sequencial compare in general
            cur_row = row.tolist()
            cur_stat = (cur_row[0], cur_row[1], cur_row[2], cur_row[3], cur_row[4], cur_row[5], cur_row[6], cur_row[7], cur_row[8])
            my_cursor.execute(sqlFormula, cur_stat)

        #commit change
        conn.commit()
        conn.close()
    

In [53]:
player_bio_table.head()

,PlayerName,Team,AvgPoints,AvgRebounds,AvgAssists,AvgSteals,AvgRatings,Season,SeasonType
0,Anthony Davis,NOP,31.5,11.0,2.0,1.3,0.0,2014-15,Playoff
1,LeBron James,CLE,30.1,11.3,8.5,1.7,0.0,2014-15,Playoff
2,Stephen Curry,GSW,28.3,5.0,6.4,1.9,0.0,2014-15,Playoff
3,James Harden,HOU,27.2,5.7,7.5,1.6,0.0,2014-15,Playoff
4,Monta Ellis,DAL,26.0,3.2,5.2,2.0,0.0,2014-15,Playoff


In [83]:
#player box score
url = 'https://stats.nba.com/players/boxscores-traditional'
browser.get(url)
option_list = [2,3,4,5,6]
season_list = ["2018-19","2017-18","2016-17","2015-16","2014-15"]

In [ ]:
#find right table
browser.find_element_by_xpath('/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[1]/div/div/label/select/option[2]').click()
time.sleep(5)
browser.find_element_by_xpath('/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[2]/div/div/label/select/option[3]').click()
time.sleep(5)

In [93]:
#get table info
table = browser.find_element_by_class_name('nba-stat-table__overflow')

#glimpse at data
'''
count = 0
for line_id, lines in enumerate(table.text.split('\n')):
    print (line_id, lines)
    count += 1
    if count >4:
        print (line_id, lines.split(' '))
        time = lines.split(' ')[4].split('/')
        print(time)
        new_time = time[-1] + "." + time[-3] + time[-2]
        new_time = float(new_time)
        print(new_time)
        break
'''

#parse table
Game_stat_table = pandas.DataFrame()
for i in range(2,37):
    action = '/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[%d]' % i
    browser.find_element_by_xpath(action).click()
    time.sleep(1)
    player_names = []
    opponent_team = []
    Points = []
    Dates = []
    Rebounds = [] 
    Assists = []
    Steals = []
    seasons = []
    season_type = []
    column_names = ['Player Name', 'Date', 'Points', 'Rebounds', 'Assists', 'Steals','Opponent Team','Season','SeasonType']
    for line_id, lines in enumerate(table.text.split('\n')):
        if line_id != 0:
            if line_id % 2 == 1:
                player_names.append(lines)
            if line_id % 2 == 0:
                temp = lines.split(' ')

                curtime = temp[4].split('/')
                new_time = curtime[-1] + curtime[-3] + curtime[-2]
                new_time = int(new_time)
                Dates.append(new_time)

                Points.append(temp[7])
                opponent_team.append(temp[3])
                Rebounds.append(temp[-7])
                Assists.append(temp[-6])
                Steals.append(temp[-5])
                seasons.append(season_list[1])
                season_type.append('Playoff')

    temp_stat_table = pandas.DataFrame({ column_names[0]: player_names,
                                       column_names[1]: Dates,
                                        column_names[2]: Points,
                                       column_names[3]: Rebounds,
                                        column_names[4]: Assists,
                                       column_names[5]: Steals,
                                        column_names[6]: opponent_team,
                                        column_names[7]: seasons,
                                        column_names[8]: season_type,
                           }
                         )
    Game_stat_table = pandas.concat([Game_stat_table, temp_stat_table], ignore_index=True)

conn = pymysql.connect(
    port=int(3306),
    user="root",
    passwd= "12345678",
    database = "NBA_DB"
)
my_cursor = conn.cursor()
#my_cursor.execute("CREATE TABLE Game_Stats (playName VARCHAR(255), Date int, Points int, Rebound int, Assists int, Steals int, OpponentTeam VARCHAR(255),Season VARCHAR(255), SeasonType VARCHAR(255))")
sqlFormula = "INSERT INTO Game_Stats (playName, Date, Points, Rebound, Assists, Steals, OpponentTeam, Season, SeasonType) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
my_cursor = conn.cursor()
for index, row in Game_stat_table.iterrows():
    #sequencial compare in general
    cur_row = row.tolist()
    cur_stat = (cur_row[0], int(cur_row[1]),cur_row[2],int(cur_row[3]), int(cur_row[4]),int(cur_row[5]),cur_row[6],cur_row[7],cur_row[8])
    my_cursor.execute(sqlFormula, cur_stat)
    
conn.commit()

In [90]:
Game_stat_table.head()

,Player Name,Date,Points,Rebounds,Assists,Steals,Opponent Team,Season,SeasonType
0,Klay Thompson,20150616,5,5,2,0,CLE,2014-15,Playoff
1,James Jones,20150616,5,1,0,0,GSW,2014-15,Playoff
2,JR Smith,20150616,19,5,1,1,GSW,2014-15,Playoff
3,Leandro Barbosa,20150616,5,3,1,2,CLE,2014-15,Playoff
4,Festus Ezeli,20150616,10,4,0,0,CLE,2014-15,Playoff


In [3]:
##player clutch stats
url = 'https://stats.nba.com/players/clutch-traditional'
browser.get(url)


In [9]:
option_list = [2,3,4,5,6]
season_list = ["2018-19","2017-18","2016-17","2015-16","2014-15"]
for season_id in range(len(option_list)):
    for season_option in range(2,4):
        season_path = '/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[1]/div/div/label/select/option[%d]' % option_list[season_id]
        browser.find_element_by_xpath(season_path).click()
        time.sleep(3)
        seasontype_path = '/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[2]/div/div/label/select/option[%d]' % season_option
        browser.find_element_by_xpath(seasontype_path).click()
        time.sleep(3)
        browser.find_element_by_xpath('/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[3]/div/div/label/select/option[1]').click()
        time.sleep(3)
        browser.find_element_by_xpath('/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]').click()
        #get table info

        table = browser.find_element_by_class_name('nba-stat-table__overflow')
        
        seasonType = ''
        if season_option == 2:
            seasonType = "Regular_Season"
        else:
            seasonType = "Playoff"

        '''
        #glimpse at data
        count = 0
        for line_id, lines in enumerate(table.text.split('\n')):
            print (line_id, lines)
            count += 1
            if count >4:
                break
        count = 0
        '''

        #parse table
        player_names = []
        ThreePointer_P = []
        FG_P = []
        FT_P = []
        minutes_played = []
        seasons = []
        season_type = []
        column_names = ['PlayerName', '3pointer%', 'Field_Goal%','Free_Throw%','Minutes_Played','Season','SeasonType']
        for line_id, lines in enumerate(table.text.split('\n')):
            if line_id != 0:
                if line_id % 3 == 2:
                    player_names.append(lines)
                if line_id % 3 == 0:
                    temp = lines.split(' ')
                    ThreePointer_P.append(temp[12])
                    FG_P.append(temp[9])
                    FT_P.append(temp[15])
                    minutes_played.append(temp[5])
                    seasons.append(season_list[season_id])
                    season_type.append(seasonType)
                    

        #create dataframe
        Player_clutch_table = pandas.DataFrame({ column_names[0]: player_names,
                                              column_names[1]: ThreePointer_P,
                                              column_names[2]: FG_P,
                                              column_names[3]: FT_P,
                                              column_names[4]: minutes_played,
                                              column_names[5]: seasons,
                                              column_names[6]: seasonType
                               }
                             )

        conn = pymysql.connect(
            port=int(3306),
            user="root",
            passwd= "12345678",
            database = "NBA_DB"
        )

        if(season_id == 0 and season_option == 2):
            my_cursor = conn.cursor()
            my_cursor.execute("CREATE TABLE Player_Clutch_Stats (playName VARCHAR(255), 3pointer_P float, Field_Goal_P float, Free_Throw_P float, Minutes_Played int, Season VARCHAR(255), SeasonType VARCHAR(255))")
        my_cursor = conn.cursor()
        sqlFormula = "INSERT INTO Player_Clutch_Stats (playName, 3pointer_P, Field_Goal_P, Free_Throw_P, Minutes_Played, Season, SeasonType) VALUES (%s,%s,%s,%s,%s,%s,%s)"
        for index, row in Player_clutch_table.iterrows():
            #sequencial compare in general
            cur_row = row.tolist()
            cur_stat = (cur_row[0], cur_row[1],cur_row[2],cur_row[3],cur_row[4],cur_row[5],cur_row[6])
            my_cursor.execute(sqlFormula, cur_stat)

        #commit change
        conn.commit()
        conn.close()

In [115]:
Player_clutch_table.head()

,PlayerName,3pointer%,Field_Goal%,Free_Throw%,Minutes_Played
0,De'Aaron Fox,37.5,44.9,80.0,156
1,D.J. Augustin,31.3,41.7,83.3,153
2,Evan Fournier,22.6,43.9,73.9,152
3,Nikola Vucevic,6.3,37.8,69.2,159
4,Tobias Harris,37.5,52.2,76.7,174


In [11]:
#Team clutch stats
url = 'https://stats.nba.com/teams/clutch-traditional'
browser.get(url)

In [ ]:
#find right table
time.sleep(5)
browser.find_element_by_xpath('/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[1]/div/div/label/select/option[2]').click()
time.sleep(5)
browser.find_element_by_xpath('/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[2]/div/div/label/select/option[2]').click()

In [20]:
#get table info
table = browser.find_element_by_class_name('nba-stat-table__overflow')

'''
#glimpse at data
count = 0
for line_id, lines in enumerate(table.text.split('\n')):
    print (line_id, lines)
    count += 1
    if count >4:
        break
'''


nbaTeams = {}
nbaTeams['Atlanta Hawks'] = 'ATL'
nbaTeams['Brooklyn Nets'] = 'BKN'
nbaTeams['Boston Celtics'] = 'BOS'
nbaTeams['Charlotte Hornets'] = 'CHA'
nbaTeams['Chicago Bulls'] = 'CHI'
nbaTeams['Cleveland Cavaliers'] = 'CLE'
nbaTeams['Dallas Mavericks'] = 'DAL'
nbaTeams['Denver Nuggets'] = 'DEN'
nbaTeams['Detroit Pistons'] = 'DET'
nbaTeams['Golden State Warriors'] = 'GSW'
nbaTeams['Houston Rockets'] = 'HOU'
nbaTeams['Indiana Pacers'] = 'IND'
nbaTeams['LA Clippers'] = 'LAC'
nbaTeams['Los Angeles Lakers'] = 'LAL'
nbaTeams['Memphis Grizzlies'] = 'MEM'
nbaTeams['Miami Heat'] = 'MIA'
nbaTeams['Milwaukee Bucks'] = 'MIL'
nbaTeams['Minnesota Timberwolves'] = 'MIN'
nbaTeams['New Orleans Pelicans'] = 'NOP'
nbaTeams['New York Knicks'] = 'NYK'
nbaTeams['Oklahoma City Thunder'] = 'OKC'
nbaTeams['Orlando Magic'] = 'ORL'
nbaTeams['Philadelphia 76ers'] = 'PHI'
nbaTeams['Phoenix Suns'] = 'PHX'
nbaTeams['Portland Trail Blazers'] = 'POR'
nbaTeams['Sacramento Kings'] = 'SAC'
nbaTeams['San Antonio Spurs'] = 'SAS'
nbaTeams['Toronto Raptors'] = 'TOR'
nbaTeams['Utah Jazz'] = 'UTA'
nbaTeams['Washington Wizards'] = 'WAS'

#parse table
Team_names = []
FG_P = []
column_names = ['TeamName', 'Field_Goal%']
for line_id, lines in enumerate(table.text.split('\n')):
    if line_id != 0:
        if line_id % 3 == 2:
            Team_names.append(nbaTeams[lines])
        if line_id % 3 == 0:
            temp = lines.split(' ')
            FG_P.append(temp[8])
            
            
#create dataframe
Team_clutch_table = pandas.DataFrame({ column_names[0]: Team_names,
                                      column_names[1]: FG_P,
                       }
                     )

#upload to sql server
conn = pymysql.connect(
    port=int(3306),
    user="root",
    passwd= "12345678",
    database = "NBA_DB"
)

my_cursor = conn.cursor()
my_cursor.execute("CREATE TABLE Team_Clutch_Stats (TeamName VARCHAR(255), Field_Goal_P float)")
sqlFormula = "INSERT INTO Team_Clutch_Stats (TeamName, Field_Goal_P) VALUES (%s,%s)"
my_cursor = conn.cursor()
for index, row in Team_clutch_table.iterrows():
    #sequencial compare in general
    cur_row = row.tolist()
    cur_stat = (cur_row[0], cur_row[1])
    my_cursor.execute(sqlFormula, cur_stat)

#commit change
conn.commit()
conn.close()

In [21]:
Team_clutch_table.head()

,TeamName,Field_Goal%
0,DEN,45.4
1,PHI,44.8
2,LAC,48.2
3,MIL,46.8
4,SAS,47.3


In [26]:
option_list = [2,3,4,5,6]
season_list = ["2018-19","2017-18","2016-17","2015-16","2014-15"]
for season_id in range(len(option_list)):
    season_path = '/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[1]/div/div/label/select/option[%d]' % option_list[season_id]
    browser.find_element_by_xpath(season_path).click()
    time.sleep(3)
    seasontype_path = '/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[2]/div/div/label/select/option[1]'
    browser.find_element_by_xpath(seasontype_path).click()
    time.sleep(3)
    browser.find_element_by_xpath('/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[3]/div/div/label/select/option[1]').click()
    time.sleep(3)
    browser.find_element_by_xpath('/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]').click()
    time.sleep(3)
    
    #get table info
    table = browser.find_element_by_class_name('nba-stat-table__overflow')

    #glimpse at data
    count = 0
    for line_id, lines in enumerate(table.text.split('\n')):
        #print (line_id, lines)
        count += 1
        if count >12:
            break

    #parse table 2FGA, 2FG%, 3FGA, 3FG%
    player_names = []
    Two_FG_A = []
    Two_FG_P = []
    Three_FG_A = []
    Three_FG_P = []
    Teams = []
    Seasons = []
    column_names = ['PlayerName', "2FGA", "2FG%", "3FGA", "3FG%","Team","Season"]
    for line_id, lines in enumerate(table.text.split('\n')):
        if line_id > 2:
            if line_id % 2 == 1:
                player_names.append(lines)
            if line_id % 2 == 0:
                temp = lines.split(' ')
                Two_FG_A.append(temp[-6])
                if(temp[-5] == "-"):
                    Two_FG_P.append(0)
                else:
                    Two_FG_P.append(temp[-5])

                if(temp[-1] == "-"):
                    Three_FG_P.append(0)
                else:
                    Three_FG_P.append(temp[-1])

                Three_FG_A.append(temp[-2])
                Teams.append(temp[0])
                Seasons.append(season_list[season_id])

    #create dataframe
    Shot_selection_table = pandas.DataFrame({ column_names[0]: player_names,
                           column_names[1]: Two_FG_A,
                                         column_names[2]: Two_FG_P,
                           column_names[3]: Three_FG_A,
                                         column_names[4]: Three_FG_P,
                            column_names[5]: Teams,
                                             column_names[6]: Seasons
                           }
                         )

    #write into mysql server
    conn = pymysql.connect(
        port=int(3306),
        user="root",
        passwd= "12345678",
        database = "NBA_DB"
    )
    if season_id == 0:
        my_cursor = conn.cursor()
        my_cursor.execute("CREATE TABLE Shot_Selection (PlayerName VARCHAR(255), 2FGA int, 2FG_P float, 3FGA int, 3FG_P float, Team VARCHAR(255), Season VARCHAR(255))")

    my_cursor = conn.cursor()
    sqlFormula = "INSERT INTO Shot_Selection (PlayerName, 2FGA, 2FG_P, 3FGA, 3FG_P, Team, Season) VALUES (%s, %s, %s, %s, %s, %s, %s)"
    for index, row in Shot_selection_table.iterrows():
        #sequencial compare in general
        cur_row = row.tolist()
        cur_stat = (cur_row[0], cur_row[1], cur_row[2], cur_row[3], cur_row[4], cur_row[5], cur_row[6])
        my_cursor.execute(sqlFormula, cur_stat)

    #commit change
    conn.commit()
    conn.close()

In [3]:
from pymongo import MongoClient

In [4]:
client = MongoClient("mongodb+srv://Dennis_97:<change To Your Password>@cluster0-a55xg.mongodb.net/test?retryWrites=true&w=majority")

db = client.get_database('NBAMongo_db')

records = db.Team_Roaster

In [11]:
#new chrome browser
browser = webdriver.Chrome()

In [38]:
nbaTeams = {}
nbaTeams['Atlanta Hawks'] = 'ATL'
nbaTeams['Brooklyn Nets'] = 'BKN'
nbaTeams['Boston Celtics'] = 'BOS'
nbaTeams['Charlotte Hornets'] = 'CHA'
nbaTeams['Chicago Bulls'] = 'CHI'
nbaTeams['Cleveland Cavaliers'] = 'CLE'
nbaTeams['Dallas Mavericks'] = 'DAL'
nbaTeams['Denver Nuggets'] = 'DEN'
nbaTeams['Detroit Pistons'] = 'DET'
nbaTeams['Golden State Warriors'] = 'GSW'
nbaTeams['Houston Rockets'] = 'HOU'
nbaTeams['Indiana Pacers'] = 'IND'
nbaTeams['LA Clippers'] = 'LAC'
nbaTeams['Los Angeles Lakers'] = 'LAL'
nbaTeams['Memphis Grizzlies'] = 'MEM'
nbaTeams['Miami Heat'] = 'MIA'
nbaTeams['Milwaukee Bucks'] = 'MIL'
nbaTeams['Minnesota Timberwolves'] = 'MIN'
nbaTeams['New Orleans Pelicans'] = 'NOP'
nbaTeams['New York Knicks'] = 'NYK'
nbaTeams['Oklahoma City Thunder'] = 'OKC'
nbaTeams['Orlando Magic'] = 'ORL'
nbaTeams['Philadelphia 76ers'] = 'PHI'
nbaTeams['Phoenix Suns'] = 'PHX'
nbaTeams['Portland Trail Blazers'] = 'POR'
nbaTeams['Sacramento Kings'] = 'SAC'
nbaTeams['San Antonio Spurs'] = 'SAS'
nbaTeams['Toronto Raptors'] = 'TOR'
nbaTeams['Utah Jazz'] = 'UTA'
nbaTeams['Washington Wizards'] = 'WAS'
nbaTeams_cap = {}
for key in nbaTeams:
    nbaTeams_cap[key.upper()] = nbaTeams[key]

option_list = [1,2,3,4,5,6]
season_list = ["2019-20","2018-19","2017-18","2016-17","2015-16","2014-15"]
for i in range(37,67):
    url = 'https://stats.nba.com/team/16106127%d/' % i
    browser.get(url)
    time.sleep(15)
    for season_id in range(len(option_list)):
        display_path = '/html/body/main/div[2]/div/div/div[3]/div/div/div[1]/div[1]/div/div/div/label/select/option[%d]' % option_list[season_id]
        browser.find_element_by_xpath(display_path).click()
        time.sleep(3)

        table = browser.find_element_by_class_name('nba-stat-table__overflow')

        #parse table
        player_names = []
        for line_id, lines in enumerate(table.text.split('\n')):
            if line_id % 2 == 1:
                player_names.append(lines)

        City = browser.find_element_by_class_name('stats-team-summary__city')
        Name = browser.find_element_by_class_name("stats-team-summary__name")
        TeamName =  City.text + " "+ Name.text

        new_Team = {
           'name': nbaTeams_cap[TeamName],
        'Players': player_names,
        'Season': season_list[season_id]
        }
        records.insert_one(new_Team)
    print(i)

62
63
64
65
66
